In [5]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import import_ipynb

from Experiment_final import get_dataset, get_q_batches, get_q_ests


ImportError: cannot import name 'get_dataset'

In [ ]:
def plt_stripe(tau_lst, ax, q_lst, pattern, name):
    hist_on = (len(q_lst.shape) > 2)
    q_lst = q_lst.T
    for idx, q in enumerate(q_lst):
        colorVal = scalarMap.to_rgba(tau_lst[idx])
        lb = str(+tau_lst[idx])+name
        if not hist_on:
            line = ax.plot([q,q], [0,1], pattern, color=colorVal)
            line[0].set_label(lb)
        else:
            bins = 30
            line = ax.hist(q.reshape(-1), bins, color=colorVal, label=lb)

In [ ]:
def plot_spectrum_stripes(q_lst, q_batches, q_sgd_res):
    c_Norm = colors.Normalize(vmin=0, vmax=1)
#     scalarMap = cmx.ScalarMappable(norm=c_Norm, cmap=plt.get_cmap('cool'))
    styles = ['-', '--']
    names = ['-q batch', '-q sgd   ']
    
    fig = plt.figure(figsize=(16,4))
    ax_batch = fig.add_subplot(211)
    ax_sgd = fig.add_subplot(212, sharex = ax_batch)
    fig.suptitle('Batch quantile VS SGD quantile')
    
    plt_stripe(q_lst, ax_batch, q_batches, styles[0], names[0])
    plt_stripe(q_lst, ax_sgd, q_sgd_res, styles[1], names[1])

    #set 
    ax_batch.set_xlabel('batch value')
    ax_batch.xaxis.set_label_coords(0.04, 0.98)
    ax_batch.set_yticks([])

    plt.locator_params(axis='x', nbins=10)

    ax_sgd.set_xlabel('sgd value')
    ax_sgd.xaxis.set_label_coords(0.035, 0.98)
    ax_sgd.set_yticks([])


    # set position of legend
    ax_batch.legend(loc='lower center', bbox_to_anchor=(0.5, -1.8),
            frameon=False, ncol=len(q_lst))
    ax_sgd.legend(loc='lower center', bbox_to_anchor=(0.5, -0.8),
        frameon=False, ncol=len(q_lst))
        
    return fig

# plot_spectrum_stripes(tau_vals, q_batches, q_sgd_res)

In [ ]:
# q_lst = tau_vals
# dataset, q_batches = generate_reusable_data(distro=distro_list[0], datasize=5000, q_lst=q_lst,
#                                             g_test=True, s_test=False)
# q_sgd_res, proc_filename, proc_shape = generate_q_sgds(dataset, stepsize_list[0], q_lst)
# print (q_batches.shape)
# # print (q_sgd_res.T.shape)
# q_procs = np.memmap(proc_filename, np.float32, 'r+', shape=proc_shape)
# q_procs.shape
# # q_batches.mean(axis=0)

In [ ]:
def plot_one_proc(tau_lst, q_proc, ax, label_on):
    for idx, q in enumerate(q_proc):
        colorVal = scalarMap.to_rgba(tau_lst[idx])
        lb = str(+tau_lst[idx])+'-quantile' if label_on else ''
        ax.plot(q, color=colorVal, label=lb)      

def plot_procs(tau_lst, q_batches, q_procs):

    fig = plt.figure(figsize=(16, 8))
    fig.suptitle('SGD Quantile Estimation for ')
    ax = fig.add_subplot(111)
    N_dim = len(q_procs.shape)

    if q_batches is not None:
        if len(q_batches.shape)==2: q_batches = q_batches.mean(axis=0)
        for idx, q in enumerate(q_batches):
            colorVal = scalarMap.to_rgba(tau_lst[idx])                
            ax.axhline(q, color=colorVal)
            
    if N_dim == 2:
        plot_one_proc(tau_lst, q_procs, ax, True)
            
    elif N_dim == 3: 
        for i, q_dt in enumerate(q_procs):
            label_on = True if i == 0 else False
            plot_one_proc(tau_lst, q_dt, ax, label_on)

    elif N_dim > 3:
        raise  Exception ("q_sgd_proc too much to be printed")

    ax.legend(loc='right', bbox_to_anchor=(1.2, 0.5),
        frameon=False, ncol=1)
    ax.set_xlabel('Number of Samples')
    if (q_batches is not None):
        ax.set_ylabel('Quantile Value')  
    else: ax.set_ylabel('Error Value')
    return fig

# plot_procs(q_lst, q_batches, q_procs)

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def plot_procs_error_bar(tau_lst, q_batches, q_procs):

    fig = plt.figure(figsize=(16, 8))
    fig.suptitle('SGD Quantile Estimation for ')
    ax = fig.add_subplot(111)
    N_dim = len(q_procs.shape)

    if q_batches is not None:
        if len(q_batches.shape)==2: q_batches = q_batches.mean(axis=0)
        for idx, q in enumerate(q_batches):
            colorVal = scalarMap.to_rgba(tau_lst[idx])                
            ax.axhline(q, color=colorVal)
            
    if N_dim == 2:
        plot_one_proc(tau_lst, q_procs, ax, True)
            
    elif N_dim == 3: 
        proc_mean = np.mean(q_procs, 0)
        proc_max = np.max(q_procs, 0) - proc_mean
        proc_min = proc_mean - np.min(q_procs, 0) 
        
        for i in range(len(tau_lst)):
            
            err = np.vstack((proc_max[i], proc_min[i]))
            colorVal = scalarMap.to_rgba(tau_lst[i])
            ax.errorbar(np.arange(proc_mean.shape[-1]),
                        proc_mean[i], 
                        yerr=err, ecolor = lighten_color(colorVal, 0.4),
                        color=colorVal, label = str(tau_lst[i])+'quantile')

    elif N_dim > 3:
        raise  Exception ("q_sgd_proc too much to be printed")

    ax.legend(loc='right', bbox_to_anchor=(1.2, 0.5),
        frameon=False, ncol=1)
    ax.set_xlabel('Number of Samples')
    if (q_batches is not None):
        ax.set_ylabel('Quantile Value')  
    else: ax.set_ylabel('Error Value')
    return fig

# plot_procs_error_bar(q_lst, q_batches, q_procs)

In [ ]:
def compute_E_res(tau_lst, q_batches, q_sgd_res):
#     if len(q_batches.shape)==len(q_sgd_res.shape):
    if len(q_sgd_res.shape) < 3:    
        e = q_sgd_res - q_batches
    else:
        e = np.zeros(q_sgd_res.shape)
        for i, q_res_g in enumerate(q_sgd_res):
            e[i] = q_res_g - q_batches[i]
    return e
    
# e = compute_E_res(q_lst, q_batches, q_sgd_res)

def plot_E_res(tau_lst, e):
    e = e.T
    if len(e.shape)>2: raise Exception('E shape is too high!')
    fig, axes = plt.subplots(nrows=len(tau_lst), sharex=True)
    fig.set_size_inches(14, 8)
    for i, ax in enumerate(axes):
        colorVal = scalarMap.to_rgba(tau_lst[i])
        ax.hist(e[i], bins=20, color=colorVal)
        ax.plot([e[i].mean(), e[i].mean()], [0,2], color='black')
        ax.set_xlabel((str(tau_lst[i])+'-quantile'))
        ax.xaxis.set_label_coords(0.11, 0.97)
#         ax.plot([e[i].mean(), e[i].mean()], [0,1], color='black')
    fig.suptitle('Error value distributions for different quantiles')
    return fig

# plot_E_res(q_lst, e)

In [ ]:
# def compute_E_proc(q_batches, q_procs):
#     q_batches = np.expand_dims(q_batches, len(q_batches.shape))
#     return q_procs - q_batches
# #     return

# e_procs = compute_E_proc(q_batches, q_procs)
# print (e_procs.shape)
# plot_procs_error_bar(q_lst, None, e_procs)

In [ ]:
def plot_charts(folder_name, q_lst, q_batches, q_sgd_res, proc_filename, proc_shape):
    
    print ("compare results with stripe plots")
    stripes = plot_spectrum_stripes(q_lst, q_batches, q_sgd_res)
    plt.savefig( folder_name+"res_stripes.png")
    
    print ("compare processes with error bars")
    q_procs = np.memmap(proc_filename, np.float32, 'r+', shape=proc_shape)
    errbars = plot_procs_error_bar(q_lst, q_batches, q_procs)
    plt.savefig(folder_name+"proc_errbars.png")
    
    print ("compare E results")
    e = compute_E_res(q_lst, q_batches, q_sgd_res)
    errs = plot_E_res(q_lst, e)
    plt.savefig(folder_name+"e_stripes.png")
    
#     return plts
    return 4

def plot_tables(q_lst, q_batches, q_sgd_res, proc_filename, proc_shape, g_test, s_test, charts):
#     return tbls
    return 5

In [ ]:
# # Generate data to test plots
# # dataset = generate_dataset(distro=distro_list[0], datasize=1000, g_test=False, s_test=False)
# dataset, q_batches = generate_reusable_data(distro=distro_list[2], datasize=1000, q_lst=tau_vals,
#                                             g_test=True, s_test=False)
# q_sgd_res, proc_filename, proc_shape = generate_q_sgds(dataset, stepsize_list[0], tau_vals)
# # print (q_batches)
# print (q_sgd_res.T.shape)

# directory = "trail/"+"something/"
# if not os.path.exists(directory):
#     os.makedirs(directory)
# plot_charts(directory, q_lst, q_batches, q_sgd_res, proc_filename, proc_shape)